# 2nd Level Model Structure: Reactive Stage

In [1]:
import sys
import os
import pickle
sys.path.append(os.path.abspath('..'))
import numpy as np
from matplotlib import pyplot as plt

In [2]:
from pyomo import environ as pe
from global_sets.component import m
from utility.display_utility import trans_product_mole, trans_product_mass
from utility.model_utility import add_dual, update_dual, check_DOF

# stage construction rules
from physics.kinetics_bounded import kinetic_block_rule
from physics.energy_bounded import energy_block_rule
from physics.VLE_bounded import VLE_block_rule
from physics.MPCC_T import dew_block_rule

model = pe.ConcreteModel()

# Global Sets (Inlet / Outlet)

In [3]:
model.inlet = pe.Set(initialize=['in'])
model.outlet = pe.Set(initialize=['out','P'])
model.stream = model.inlet | model.outlet

# Global Variables

In [4]:
# Tray Inlet/Outlet Variable
model.x_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y_ = pe.Var(model.inlet,m.COMP_TOTAL,within=pe.NonNegativeReals)
model.x = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.y = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals)
model.z = pe.Var(m.COMP_FEED,within=pe.NonNegativeReals)

model.L = pe.Var(model.stream,within=pe.NonNegativeReals)
model.V = pe.Var(model.stream,within=pe.NonNegativeReals)
model.F = pe.Var(within=pe.NonNegativeReals)

model.H_L_ = pe.Var(model.inlet,within=pe.Reals)
model.H_V_ = pe.Var(model.inlet,within=pe.Reals)
model.H_L = pe.Var(within=pe.Reals)
model.H_V = pe.Var(within=pe.Reals)
model.H_F = pe.Var(within=pe.Reals)

# State Variable
model.T = pe.Var(within=pe.NonNegativeReals,bounds=(200+273.15,300+273.15)) # K
model.T_F = pe.Var(within=pe.NonNegativeReals) # K
model.P = pe.Var(within=pe.NonNegativeReals,bounds=(10,30)) # Bar

model.f_V = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)
model.f_L = pe.Var(m.COMP_TOTAL,within=pe.NonNegativeReals,initialize=1e-20)

model.cat = pe.Var(within=pe.NonNegativeReals) # kg
model.Q_main = pe.Var(within=pe.Reals) # MW
model.r_total_comp = pe.Var(m.COMP_TOTAL,within=pe.Reals) # kmol/s

# Construct Individual Blocks

In [5]:
model.kinetics_block = pe.Block(rule=kinetic_block_rule)

> Importing Kinetics Blocks......
> Adding the following local variable:
--------------------------------------------------
| kinetics_block.k_FT
| kinetics_block.r_FT_total
| kinetics_block.g0_FT
| kinetics_block.alpha
| kinetics_block.r_FT_cnum
| kinetics_block.r_FT_comp
| kinetics_block.k_WGS
| kinetics_block.Ke_WGS
| kinetics_block.r_WGS
| kinetics_block.r_WGS_comp
--------------------------------------------------



In [6]:
model.energy_block = pe.Block(rule=energy_block_rule)

> Importing Energy Blocks......
> Adding the following local variable:
--------------------------------------------------
| energy_block.dH_F
| energy_block.dH_V
| energy_block.dH_L
| energy_block.dH_vap
--------------------------------------------------



In [7]:
model.VLE_block = pe.Block(rule=VLE_block_rule)

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| VLE_block.n_ave
| VLE_block.n_ave_cal
| VLE_block.Hen
| VLE_block.Hen0
| VLE_block.gamma
| VLE_block.P_sat
| VLE_block.P_sat_Y
| VLE_block.P_sat_dY_inf
| VLE_block.P_sat_dY0
| VLE_block.Hen_ref
| VLE_block.Hen0_ref
| VLE_block.gamma_ref
| VLE_block.V_L
| VLE_block.V_L_dY_inf
| VLE_block.V_L_dY0
| VLE_block.poynting
--------------------------------------------------



In [8]:
model.MPCC = pe.Block(rule=dew_block_rule)

> Importing dew Blocks......
> Adding the following local variable:
------------------------------------
| MPCC.x
| MPCC.y
| MPCC.T
| MPCC.P
| MPCC.f_V
| MPCC.f_L
------------------------------------

> Importing VLE Blocks......
> Adding the following local variable:
--------------------------------------------------
| MPCC.VLE_block.n_ave
| MPCC.VLE_block.n_ave_cal
| MPCC.VLE_block.Hen
| MPCC.VLE_block.Hen0
| MPCC.VLE_block.gamma
| MPCC.VLE_block.P_sat
| MPCC.VLE_block.P_sat_Y
| MPCC.VLE_block.P_sat_dY_inf
| MPCC.VLE_block.P_sat_dY0
| MPCC.VLE_block.Hen_ref
| MPCC.VLE_block.Hen0_ref
| MPCC.VLE_block.gamma_ref
| MPCC.VLE_block.V_L
| MPCC.VLE_block.V_L_dY_inf
| MPCC.VLE_block.V_L_dY0
| MPCC.VLE_block.poynting
--------------------------------------------------



# Standard MESH Equations

## Mass Balance

\begin{equation}
Fz_{i}+\sum_{s\in{inlet}}L_{s}x'_{i,s}+\sum_{s\in{inlet}}V_{s}y'_{i,s}-\sum_{s\in{outlet}}L_{s}x_{i,s}+\sum_{s\in{ouelet}}V_{s}y_{i,s}+R_{i} = 0 \\
i = 1,...NC
\end{equation}

In [9]:
def mass_balance_main_rule(model,i):
    if i in m.COMP_FEED:
        return model.F*model.z[i] + sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
    else:
        return sum(model.L[s]*model.x_[s,i] + model.V[s]*model.y_[s,i] for s in model.inlet)\
        + model.r_total_comp[i] - sum(model.L[s]*model.x[i] + model.V[s]*model.y[i] for s in model.outlet) == 0
model.mass_balance_main_con = pe.Constraint(m.COMP_TOTAL,rule=mass_balance_main_rule)

## Equilibrium

\begin{align}
&f_{i,V} = f_{i,L} \\
&i = 1,...NC
\end{align}

In [10]:
def VL_equil_rule(model,i):
    return model.f_V[i] == model.f_L[i]
model.VL_equil_con = pe.Constraint(m.COMP_TOTAL,rule=VL_equil_rule)

## Summation

\begin{equation}
\sum_ix_{i} = \sum_iy_{i} \\
F + L_{in} + V_{in} + R = L_{out} + V_{out}
\end{equation}

In [11]:
def summation_x_y_rule(model):
    return sum(model.x[i] for i in m.COMP_TOTAL) == sum(model.y[i] for i in m.COMP_TOTAL)
model.summation_x_y_con = pe.Constraint(rule=summation_x_y_rule)

def summation_total_mass_rule(model):
    return model.F + sum(model.L[s] + model.V[s] for s in model.inlet) + sum(model.r_total_comp[i] for i in m.COMP_TOTAL)\
            - sum(model.L[s] + model.V[s] for s in model.outlet) == 0
model.summation_total_mass_con = pe.Constraint(rule=summation_total_mass_rule)

## Energy Balance

\begin{equation}
F H_f+\sum_{s\in{inlet}}L_{s}H_{l,s}+\sum_{s\in{inlet}}V_{s}H_{v,s}-\sum_{s\in{outlet}}L_{s}H_{l,s}-\sum_{s\in{outlet}}V_{s}H_{v,s}+Q = 0
\end{equation}

In [12]:
def heat_balance_main_rule(model):
    return model.F*model.H_F + sum(model.L[s]*model.H_L_[s] + model.V[s]*model.H_V_[s] for s in model.inlet) \
            + model.Q_main - sum(model.L[s]*model.H_L + model.V[s]*model.H_V for s in model.outlet) == 0
model.heat_balance_main_con = pe.Constraint(rule=heat_balance_main_rule)

# Testing

In [13]:
add_dual(pe,model)

Created the follow pyomo suffixes:
ipopt_zL_out, ipopt_zU_out, ipopt_zL_in, ipopt_zU_in, dual


## Fixing In/Product Redundent Flow

In [14]:
model.x_.fix(0)
model.y_.fix(0)
model.L['in'].fix(0)
model.L['P'].fix(0)
model.V['in'].fix(0)
model.V['P'].fix(0)
model.H_L_.fix(0)
model.H_V_.fix(0)

## Fixing Model Parameters

In [15]:
model.cat.fix(300)
model.P.fix(20)
model.T_F.fix(200+273.15)
model.F.fix(1)
model.z['CO'].fix(1/(1+0.8)-0.01/2)
model.z['H2'].fix(0.8/(1+0.8)-0.01/2)
model.z['C30H62'].fix(0.01)
model.VLE_block.n_ave.fix(20)

In [16]:
check_DOF(pe,model)

Active Equality Constraints:	 1893
Active Inequality Constraints:	 0
Active Variables:		 2067
Fixed Variables:		 173
DOF:				 1


## Fix T or to fix Q? we have found that Maximize against an upper bound is most reliable

In [17]:
model.T.setub(220+273.15)

In [18]:
# model.obj = pe.Objective(expr = model.L['out'],sense=pe.maximize)
# model.obj = pe.Objective(expr = model.Q_main,sense=pe.maximize)
model.obj = pe.Objective(expr = model.T,sense=pe.maximize)

In [19]:
opt = pe.SolverFactory('ipopt')

opt.options['print_user_options'] = 'yes'
# opt.options['linear_solver'] = 'ma97'
opt.options['linear_system_scaling '] = 'mc19'
opt.options['linear_scaling_on_demand '] = 'no'
# opt.options['nlp_scaling_method'] = None
# opt.options['bound_relax_factor'] = 0
# opt.options['halt_on_ampl_error'] = 'yes'

opt.options['max_iter'] = 7000
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_system_scaling =mc19
linear_scaling_on_demand =no
max_iter=7000


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 7000                  yes
                      print_user_options = yes                   yes

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt docu

  72r-4.9271035e+02 1.40e-01 9.99e+02  -0.9 0.00e+00    -  0.00e+00 2.04e-07R  2
  73r-4.9271096e+02 1.38e-01 9.98e+02  -0.9 2.20e+02    -  6.60e-02 1.58e-03f  1
  74r-4.9271721e+02 1.59e-01 9.75e+02  -0.9 1.90e+02    -  1.78e-01 2.25e-02f  1
  75r-4.9277230e+02 3.30e+00 7.96e+02  -0.9 1.16e+02    -  1.95e-01 1.84e-01f  1
  76r-4.9281025e+02 2.90e+00 1.05e+03  -0.9 4.20e+01    -  5.02e-01 1.23e-01f  1
  77r-4.9296472e+02 1.76e+00 9.39e+02  -0.9 5.33e+01    -  6.31e-01 4.31e-01f  1
  78r-4.9306336e+02 7.38e-01 8.37e+02  -0.9 9.01e+01    -  9.56e-01 5.87e-01f  1
  79r-4.9306321e+02 7.52e-02 7.60e+00  -0.9 8.36e-01   0.0 9.91e-01 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  80r-4.9306321e+02 7.52e-02 9.99e+02  -1.0 0.00e+00    -  0.00e+00 2.78e-07R  6
  81r-4.9306362e+02 7.52e-02 2.15e+03  -1.0 4.45e+01    -  7.42e-01 3.27e-02f  1
  82r-4.9305258e+02 8.40e-02 9.37e+02  -1.0 5.52e+01    -  7.96e-01 6.64e-01f  1
  83r-4.9305256e+02 7.56e-02

 338r-4.9080105e+02 7.98e-03 3.77e-04  -8.6 3.05e-02  -1.9 1.00e+00 1.00e+00h  1
 339r-4.9080105e+02 7.98e-03 3.77e-04  -8.6 1.14e-02  -1.5 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 340r-4.9080102e+02 7.98e-03 3.77e-04  -8.6 1.03e-01  -2.4 1.00e+00 1.00e+00h  1
 341r-4.9080102e+02 7.98e-03 3.77e-04  -8.6 4.83e-03  -1.1 1.00e+00 1.00e+00h  1
 342r-4.9080102e+02 7.98e-03 3.77e-04  -8.6 1.45e-02  -1.6 1.00e+00 1.00e+00h  1
 343r-4.9080101e+02 7.98e-03 3.77e-04  -8.6 1.63e-02  -1.6 1.00e+00 1.00e+00h  1
 344r-4.9080097e+02 7.98e-03 3.77e-04  -8.6 1.47e-01  -2.6 1.00e+00 1.00e+00h  1
 345r-4.9080093e+02 7.98e-03 3.77e-04  -8.6 1.65e-01  -2.6 1.00e+00 1.00e+00h  1
 346r-4.9080054e+02 7.98e-03 3.77e-04  -8.6 1.49e+00  -3.6 1.00e+00 1.00e+00h  1
 347r-4.9080039e+02 7.97e-03 3.77e-04  -8.6 5.57e-01  -3.2 1.00e+00 1.00e+00h  1
 348r-4.9079906e+02 7.97e-03 3.78e-04  -8.6 5.02e+00  -4.1 1.00e+00 1.00e+00h  1
 349r-4.9079905e+02 7.97e-03

 438r-4.8456493e+02 7.72e-03 3.67e-06  -8.6 5.96e-02  -4.2 1.00e+00 1.00e+00h  1
 439r-4.8440606e+02 9.38e-03 2.42e-03  -8.6 2.07e+01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 440r-4.8440601e+02 7.72e-03 5.99e-05  -8.6 2.57e-03    -  1.00e+00 1.00e+00h  1
 441r-4.8440302e+02 7.72e-03 1.05e-04  -8.6 3.88e-01  -5.2 1.00e+00 1.00e+00h  1
 442r-4.8439866e+02 7.72e-03 3.44e-06  -8.6 5.66e-01  -5.2 1.00e+00 1.00e+00h  1
 443r-4.7842714e+02 1.31e+01 3.65e+00  -8.6 7.76e+02  -5.7 1.00e+00 1.00e+00h  1
 444r-4.7846058e+02 4.38e+00 1.95e+02  -8.6 1.94e+01    -  5.48e-01 6.67e-01h  1
 445r-4.7848150e+02 7.73e-03 2.99e-01  -8.6 7.00e+00  -5.3 1.00e+00 1.00e+00h  1
 446r-4.7848151e+02 7.73e-03 1.01e-03  -8.6 3.16e-04    -  1.00e+00 1.00e+00h  1
 447r-4.7848151e+02 7.73e-03 2.69e+01  -8.6 6.97e+01    -  1.00e+00 1.91e-06h 20
 448r-4.7848151e+02 7.73e-03 2.07e-04  -8.6 4.61e-04  -6.2 1.00e+00 1.00e+00h  1
 449r-4.7848151e+02 7.73e-03

	message from solver=Ipopt 3.12.8\x3a Converged to a locally infeasible point. Problem may be infeasible.


In [20]:
model.MPCC.pprint()

MPCC : Size=1, Index=None, Active=True
    6 Var Declarations
        P : Size=1, Index=None
            Key  : Lower : Value : Upper : Fixed : Stale : Domain
            None :    10 :  20.0 :    30 : False : False : NonNegativeReals
        T : Size=1, Index=None
            Key  : Lower  : Value  : Upper  : Fixed : Stale : Domain
            None : 473.15 : 573.15 : 573.15 : False : False : NonNegativeReals
        f_L : Size=79, Index=COMP_TOTAL
            Key     : Lower : Value                  : Upper : Fixed : Stale : Domain
             C10H20 :     0 :    0.00204421669095706 :  None : False : False : NonNegativeReals
             C10H22 :     0 :   0.001202480402826146 :  None : False : False : NonNegativeReals
             C11H22 :     0 :  0.0012041405805587853 :  None : False : False : NonNegativeReals
             C11H24 :     0 :  0.0017972247611027401 :  None : False : False : NonNegativeReals
             C12H24 :     0 :  0.0011131515034196373 :  None : False : False

            Key     : Lower : Body                                  : Upper : Active
             C10H20 :   0.0 :   MPCC.f_V[C10H20] - MPCC.f_L[C10H20] :   0.0 :   True
             C10H22 :   0.0 :   MPCC.f_V[C10H22] - MPCC.f_L[C10H22] :   0.0 :   True
             C11H22 :   0.0 :   MPCC.f_V[C11H22] - MPCC.f_L[C11H22] :   0.0 :   True
             C11H24 :   0.0 :   MPCC.f_V[C11H24] - MPCC.f_L[C11H24] :   0.0 :   True
             C12H24 :   0.0 :   MPCC.f_V[C12H24] - MPCC.f_L[C12H24] :   0.0 :   True
             C12H26 :   0.0 :   MPCC.f_V[C12H26] - MPCC.f_L[C12H26] :   0.0 :   True
             C13H26 :   0.0 :   MPCC.f_V[C13H26] - MPCC.f_L[C13H26] :   0.0 :   True
             C13H28 :   0.0 :   MPCC.f_V[C13H28] - MPCC.f_L[C13H28] :   0.0 :   True
             C14H28 :   0.0 :   MPCC.f_V[C14H28] - MPCC.f_L[C14H28] :   0.0 :   True
             C14H30 :   0.0 :   MPCC.f_V[C14H30] - MPCC.f_L[C14H30] :   0.0 :   True
             C15H30 :   0.0 :   MPCC.f_V[C15H30] - MPCC.f_L[C15H3

            Key  : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

                    Key     : Lower : Body                                                                                                                                     : Upper : Active
                     C10H20 :   0.0 :           -2.72709 + MPCC.VLE_block.P_sat_Y[C10H20] - 8.718595*MPCC.VLE_block.P_sat_dY_inf + 0.2175393206832472*MPCC.VLE_block.P_sat_dY0 :   0.0 :   True
                     C10H22 :   0.0 :           -2.72709 + MPCC.VLE_block.P_sat_Y[C10H22] - 8.873769*MPCC.VLE_block.P_sat_dY_inf + 0.2151069188297999*MPCC.VLE_block.P_sat_dY0 :   0.0 :   True
                     C11H22 :   0.0 :          -2.72709 + MPCC.VLE_block.P_sat_Y[C11H22] - 9.718595*MPCC.VLE_block.P_sat_dY_inf + 0.20272332706887297*MPCC.VLE_block.P_sat_dY0 :   0.0 :   True
                     C11H24 :   0.0 :          -2.72709 + MPCC.VLE_block.P_sat_Y[C11H24] - 9.873769*MPCC.VLE_block.P_sat_dY_inf + 0.20059394118239604*MPCC.VLE_block.P_sat_dY0 :   0.0 :   True
                     C12H24 :   0.0 :   

                    Key     : Lower : Body                                                                                                             : Upper : Active
                     C10H20 :   0.0 :  MPCC.VLE_block.V_L[C10H20] - 11.061318*MPCC.VLE_block.V_L_dY_inf + 0.0019213141132131803*MPCC.VLE_block.V_L_dY0 :   0.0 :   True
                     C10H22 :   0.0 :  MPCC.VLE_block.V_L[C10H22] - 11.388524*MPCC.VLE_block.V_L_dY_inf + 0.0019470306875109882*MPCC.VLE_block.V_L_dY0 :   0.0 :   True
                     C11H22 :   0.0 :   MPCC.VLE_block.V_L[C11H22] - 12.061318*MPCC.VLE_block.V_L_dY_inf + 0.001849739919020478*MPCC.VLE_block.V_L_dY0 :   0.0 :   True
                     C11H24 :   0.0 :  MPCC.VLE_block.V_L[C11H24] - 12.388524*MPCC.VLE_block.V_L_dY_inf + 0.0018720880769536096*MPCC.VLE_block.V_L_dY0 :   0.0 :   True
                     C12H24 :   0.0 :  MPCC.VLE_block.V_L[C12H24] - 13.061318*MPCC.VLE_block.V_L_dY_inf + 0.0017869766611063452*MPCC.VLE_block.V_L_dY0 :   0.0 :

                    Key     : Lower : Body                                                                                                                                   : Upper : Active
                     C10H20 :   0.0 :      MPCC.f_L[C10H20] - MPCC.VLE_block.gamma[C10H20] * MPCC.VLE_block.P_sat[C10H20] * MPCC.x[C10H20] * MPCC.VLE_block.poynting[C10H20] :   0.0 :   True
                     C10H22 :   0.0 :      MPCC.f_L[C10H22] - MPCC.VLE_block.gamma[C10H22] * MPCC.VLE_block.P_sat[C10H22] * MPCC.x[C10H22] * MPCC.VLE_block.poynting[C10H22] :   0.0 :   True
                     C11H22 :   0.0 :      MPCC.f_L[C11H22] - MPCC.VLE_block.gamma[C11H22] * MPCC.VLE_block.P_sat[C11H22] * MPCC.x[C11H22] * MPCC.VLE_block.poynting[C11H22] :   0.0 :   True
                     C11H24 :   0.0 :      MPCC.f_L[C11H24] - MPCC.VLE_block.gamma[C11H24] * MPCC.VLE_block.P_sat[C11H24] * MPCC.x[C11H24] * MPCC.VLE_block.poynting[C11H24] :   0.0 :   True
                     C12H24 :   0.0 :      MPCC.f_

                    Key     : Lower : Body                                                                                                                                    : Upper : Active
                     C10H20 :   0.0 :  log( MPCC.VLE_block.gamma[C10H20] ) * ( -6 + MPCC.VLE_block.n_ave ) - log( MPCC.VLE_block.gamma_ref ) * ( -10 + MPCC.VLE_block.n_ave ) :   0.0 :   True
                     C10H22 :   0.0 :  log( MPCC.VLE_block.gamma[C10H22] ) * ( -6 + MPCC.VLE_block.n_ave ) - log( MPCC.VLE_block.gamma_ref ) * ( -10 + MPCC.VLE_block.n_ave ) :   0.0 :   True
                     C11H22 :   0.0 :  log( MPCC.VLE_block.gamma[C11H22] ) * ( -6 + MPCC.VLE_block.n_ave ) - log( MPCC.VLE_block.gamma_ref ) * ( -11 + MPCC.VLE_block.n_ave ) :   0.0 :   True
                     C11H24 :   0.0 :  log( MPCC.VLE_block.gamma[C11H24] ) * ( -6 + MPCC.VLE_block.n_ave ) - log( MPCC.VLE_block.gamma_ref ) * ( -11 + MPCC.VLE_block.n_ave ) :   0.0 :   True
                     C12H24 :   0.0 :  log( M

In [21]:
model.Q_main.value

-1.4018035815785377

In [22]:
model.T.value

478.66329009963874

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reactive_flash_300C_n58.pickle','wb') as f:
#     pickle.dump(results,f)

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))

# Iterative Solve for Data Analysis

In [ ]:
update_dual(pe,model)

In [ ]:
opt.options['warm_start_init_point'] = 'yes'
opt.options['warm_start_bound_push'] = 1e-20
opt.options['warm_start_mult_bound_push'] = 1e-20
opt.options['mu_init'] = 1e-6

In [ ]:
rf_data = {}
rf_data['r'] = {}; rf_data['T'] = []; rf_data['Q'] = []; rf_data['V'] = []; rf_data['L'] = []; 
rf_data['y_CO'] = []; rf_data['y_H2'] = []; rf_data['y_CO2'] = []; rf_data['y_H2O'] = []; rf_data['r_WGS'] = []; rf_data['r_FT'] = []
rf_data['f_V_CO'] = []; rf_data['f_V_H2'] = [];

for i in model.r_total_comp:
    rf_data['r'][i] = []
    
Trange = np.arange(300+273.15,199+273.15,-2)

for Tub in Trange:
    model.T.setub(Tub)
    results = opt.solve(model,tee=False)
    update_dual(pe,model)
    # print('-'*72)
    print('Solved, Solution T = {:.2f} K\t|\tV = {:.5f}\t|\tL = {:.5f}\t|'.format(model.T.value,model.V['out'].value,model.L['out'].value))
    # print('-'*72)
    rf_data['T'].append(model.T.value)
    rf_data['Q'].append(model.Q_main.value)
    rf_data['V'].append(model.V['out'].value)
    rf_data['L'].append(model.L['out'].value)    

    for i in model.r_total_comp:
        rf_data['r'][i].append(model.r_total_comp[i].value)
    
    
    rf_data['y_H2O'].append(model.y['H2O'].value)
    rf_data['y_CO'].append(model.y['CO'].value)
    rf_data['y_H2'].append(model.y['H2'].value)
    rf_data['y_CO2'].append(model.y['CO2'].value)
    rf_data['r_WGS'].append(model.kinetics_block.r_WGS.value)
    rf_data['r_FT'].append(model.kinetics_block.r_FT_total.value)
    
    rf_data['f_V_CO'].append(model.f_V['CO'].value)
    rf_data['f_V_H2'].append(model.f_V['H2'].value)

## Some Observations

* This system has multiple solution under same Q.
* The reaction rate peaks at ~530K, due to decreasing $\alpha$, increasing light component, uses up more feed.
* Note that when 'fully' utilizing catalyst, the system seems to always require cooling.

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,9))
line1 = ax1.plot(rf_data['T'],rf_data['Q'], 'b-')
ax1.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax1.set_ylabel('Q (MW)', color='b',fontsize=18)
ax1.tick_params('y', colors='b',labelsize=18)
ax1.tick_params('x', colors='k',labelsize=18)
ax1.grid()

ax2 = ax1.twinx()
line2 = ax2.plot(rf_data['T'],rf_data['r_FT'], 'r')
line3 = ax2.plot(rf_data['T'],rf_data['r_WGS'], 'r--')
line4 = ax2.plot(rf_data['T'],rf_data['V'], 'k')
line5 = ax2.plot(rf_data['T'],rf_data['L'], 'k--')
ax2.set_ylabel('$r_{FT}$: (kmol/s Carbon Number)', color='r',fontsize=18)
ax2.tick_params('y', colors='r',labelsize=18)

ax1.legend(line1+line2+line3+line4+line5,['Q','$r_{FT}$','$r_{WGS}$','V','L'],fontsize=18,loc=1)
ax1.set_title('Heat Removal Rate and Reaction Rate vs T',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['y_CO'])
ax.plot(rf_data['T'],rf_data['y_H2'])
ax.plot(rf_data['T'],rf_data['y_CO2'])
ax.plot(rf_data['T'],rf_data['y_H2O'])

ax.legend(['$y_{CO}$','$y_{H2}$','$y_{CO2}$','$y_{H2O}$'],fontsize=18)
ax.set_title('Gas Phase Inorganic Composition vs T',fontsize=18)

ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(rf_data['T'],rf_data['r']['CO'])
ax.plot(rf_data['T'],rf_data['r']['H2'])
ax.plot(rf_data['T'],rf_data['r']['CO2'])
ax.plot(rf_data['T'],rf_data['r']['H2O'])

ax.legend(['$r_{CO}$','$r_{H2}$','$r_{CO2}$','$r_{H2O}$'],fontsize=18)
ax.set_title('Inorganic Reaction Rate vs T',fontsize=18)

ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
ax.tick_params('y', colors='k',labelsize=18)
ax.tick_params('x', colors='k',labelsize=18)
ax.grid()
plt.show()

## Product Distribution (mole)

In [ ]:
reaction_data = trans_product_mole(rf_data['r'])['unscaled']
reaction_data_scaled = trans_product_mole(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data_scaled['c1'], reaction_data_scaled['c2'],reaction_data_scaled['c3'],reaction_data_scaled['c4'],\
            reaction_data_scaled['napha'],reaction_data_scaled['gasoline'],reaction_data_scaled['diesel'],reaction_data_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mole) vs T',fontsize=18)
ax.set_ylabel('Mole Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], reaction_data['c1'], reaction_data['c2'],reaction_data['c3'],reaction_data['c4'],\
            reaction_data['napha'],reaction_data['gasoline'],reaction_data['diesel'],reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (Mole) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kmol/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

## Product Distribution (mass)

In [ ]:
mass_reaction_data = trans_product_mass(rf_data['r'])['unscaled']
mass_reaction_scaled = trans_product_mass(rf_data['r'])['scaled']

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_scaled['c1'], mass_reaction_scaled['c2'],mass_reaction_scaled['c3'],mass_reaction_scaled['c4'],\
            mass_reaction_scaled['napha'],mass_reaction_scaled['gasoline'],mass_reaction_scaled['diesel'],mass_reaction_scaled['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Product Composition (Mass) vs T',fontsize=18)
ax.set_ylabel('Mass Fraction (%)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16,9))
ax.stackplot(rf_data['T'], mass_reaction_data['c1'], mass_reaction_data['c2'],mass_reaction_data['c3'],mass_reaction_data['c4'],\
            mass_reaction_data['napha'],mass_reaction_data['gasoline'],mass_reaction_data['diesel'],mass_reaction_data['heavy'])
plt.legend(['C1','C2','C3','C4','Napha','gasoline','diesel','heavy'])
ax.set_title('Total Production Rate (mass) vs T',fontsize=18)
ax.set_ylabel('Reaction Rate (kg/s)', color='K',fontsize=18)
ax.set_xlabel('Temperature (k)', color='K',fontsize=18)
plt.show()

In [ ]:
# model.solutions.store_to(results)
# with open('../saved_solutions/reactive_flash_200C_n58.pickle','wb') as f:
#     pickle.dump(results,f)

# VLE Validation using AspenPlus

### Feed (Reactor Effluent)

In [ ]:
print('Temperature (C): \t{:.2f}\t\t'.format(model.T.value-273.15))
print('Conversion(H2+CO): \t{:.2f}\t\t'.format(1-((model.y['CO'].value+model.y['H2'].value)*model.V['out'].value)))
print('Total FLow (kmol/s): \t{:.2f}\t\t'.format(model.L['out'].value+model.V['out'].value))
print('-'*72)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:.4%}'.format((model.x[i].value*model.L['out'].value\
        +model.y[i].value*model.V['out'].value)/(model.L['out'].value+model.V['out'].value)))

### Phase Separation

In [ ]:
print('Component\t\tLiquid: {:.4f}\t\t\tVapor: {:.4f}\t\t\tlog K'.format(model.L['out'].value,model.V['out'].value))
print('-'*108)
for i in model.x:
    print('{:10s}'.format(i),'\t\t{:8.4%}\t\t\t{:8.4%}\t\t\t{:.4f}'.format(model.x[i].value,model.y[i].value,np.log10(model.y[i].value/model.x[i].value)))